# Workflow

In this chapter, we are going to wrap our workflow into a re-usable function, see how we can handle and export the extracted tabular data and finally export all these tables when we analyze all images.

We can summarize our complete workflow as the following function which takes a three channel image as input:

In [1]:
import skimage
from microfilm.microplot import microshow
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as ndi

In [3]:
def nuclei_in_out(image):
    image_nuclei = image[:,:,2]

    # filter image
    image_nuclei = skimage.filters.median(image_nuclei, skimage.morphology.disk(5))

    # create mask and clean-up
    mask_nuclei = image_nuclei > skimage.filters.threshold_otsu(image_nuclei)
    mask_nuclei = skimage.morphology.binary_closing(mask_nuclei, footprint=skimage.morphology.disk(5))
    mask_nuclei = ndi.binary_fill_holes(mask_nuclei, skimage.morphology.disk(5))

    labels_nuclei = skimage.morphology.label(mask_nuclei)

    mask_nuclei_eroded = skimage.morphology.binary_erosion(mask_nuclei, skimage.morphology.disk(10))

    labels_masked_inner = labels_nuclei * mask_nuclei_eroded
    labels_masked_outer = labels_nuclei * (1-mask_nuclei_eroded)

    measure_inner = skimage.measure.regionprops_table(
        label_image=labels_masked_inner, intensity_image=image[:,:,1],
        properties=('label', 'area', 'mean_intensity'))

    measure_outer = skimage.measure.regionprops_table(
        label_image=labels_masked_outer, intensity_image=image[:,:,1],
        properties=('label', 'area', 'mean_intensity'))
    
    return measure_inner, measure_outer


Let's verify that it works. We define here the image path as this is what we'll have in our complete analysis:

In [4]:
from pathlib import Path

image_path = Path('../data/cellaltlas/images/37367_517_E4_2.tif')

In [5]:
image = skimage.io.imread(image_path)

In [6]:
m_in, m_out = nuclei_in_out(image)

In [7]:
m_in

{'label': array([ 1,  2,  3,  4,  5,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20]),
 'area': array([18311, 29810, 40219, 60268, 29764,  6113, 31132, 31457, 52129,
        43280, 48210, 39566, 27734, 45929, 39765, 28504, 10416,  6055]),
 'mean_intensity': array([152.02031566, 158.9123113 , 127.79708595, 164.54582863,
        140.17766429, 123.8836905 , 152.55560195, 160.7047398 ,
        117.40347983, 146.85311922, 118.43835304, 151.08664004,
        123.20080046, 118.4308389 , 131.67871244, 124.31146506,
        102.63632873,  89.75739059])}

In [8]:
m_out

{'label': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20]),
 'area': array([ 3991,  7642,  7901,  9443, 11571,     6,     2,  2534,  7225,
         7274,  9345,  9922,  8309,  8019,  4756,  8122,  8316,  5587,
         2732,  2321]),
 'mean_intensity': array([141.10172889, 149.40774666, 111.07669915, 156.43979667,
        115.1012877 ,  78.33333333,  92.        ,  93.77979479,
        141.62989619, 138.7594171 ,  97.0058855 , 121.59282403,
        100.11758334, 135.15089163,  79.55424727,  95.62127555,
         93.61075036,  80.03866118,  74.02196193,  69.26281775])}

## Comments on the function

Your are entirely free to define your functions as you like. For example, above we pass an image to the function, but we could as well pass a path and the image could be imported directly in the function. Also we return the two output dictionaries, but we could also directly save them in the function. Depending on your use case you may prefer one or the other solution.

## Generality

Once property you should generally aim for is *generality*. You might want to re-use this function for another project, and so you should be careful to not *hard-code* very specific features of your dataset in your code. For example you should avoid setting a fixed threshold value in your code. One solution for this is to pass such numbers as input to the function and to assign reasonable values.

## Handling tables: Pandas

Wherever we save the output dictionaries, we have to save them as tables. The most popular way of doing this in Python is to use the Pandas package which offers such a tabular data structure called a DataFrame. 

The first operation is to transform the dictionary into a DataFrame. Luckily this is very simple, as the keys from our dictionary can juste be used to define the columns of our table:

In [9]:
import pandas as pd

out_df = pd.DataFrame(m_out)

In [10]:
out_df

,label,area,mean_intensity
0,1,3991,141.101729
1,2,7642,149.407747
2,3,7901,111.076699
3,4,9443,156.439797
4,5,11571,115.101288
5,6,6,78.333333
6,7,2,92.000000
7,8,2534,93.779795
8,9,7225,141.629896
9,10,7274,138.759417


Now we see that this offers an actual table. Note though that it isn't an interactive table, i.e. you cannot point to lines and modify them.

These tables are very useful for post-processing as they allow combined multiple dataset, compute statistics on the table, group certain rows based on criteria etc. For example with

In [11]:
out_df.mean()

label               10.50000
area              6250.90000
mean_intensity     108.18033
dtype: float64

we can compute the mean of each column in the table without having to do that manually.

Pandas also offers a simple way to export such table into classical formats like csv. We only have to specify a valid path. For this we define a folder as a ```Path``` object:

In [12]:
from pathlib import Path

export_dir = Path('../exports/')

Now we want to keep track of which file this export corresponds to. So let's recover the name or ```stem``` of the image:

In [13]:
image_path.stem

'37367_517_E4_2'

Now we can create a full path by also adding a csv extension and a suffix ```_out``` to specify that this corresponds to the membrane (outer part) intensity.

In [24]:
export_path = export_dir.joinpath(image_path.stem +'_out.csv')

Finally we can export the DataFrame or table using the ```to_csv``` method. We just us the additional options ```index=False``` to avoid storing a useless index in the csv file:

In [25]:
out_df.to_csv(export_path, index=False)

As a control, let's check the contents of the export folder:

In [26]:
list(export_dir.iterdir())

[PosixPath('../exports/37367_517_E4_2_out.csv'),
 PosixPath('../exports/.ipynb_checkpoints')]

## Execute the workflow

We can finally run our full workflow. We just have to:
- define the list of files to analyze
- run the workflow on each file
- export the tables for each image

In [27]:
folder_to_analyze = Path('../data/cellaltlas/images/')

In [28]:
files = list(folder_to_analyze.iterdir())
files

[PosixPath('../data/cellaltlas/images/categories.csv'),
 PosixPath('../data/cellaltlas/images/27985_284_E10_2.tif'),
 PosixPath('../data/cellaltlas/images/24138_196_F7_2.tif'),
 PosixPath('../data/cellaltlas/images/50546_727_A8_2.tif'),
 PosixPath('../data/cellaltlas/images/67703_1283_D7_3.tif'),
 PosixPath('../data/cellaltlas/images/47549_736_E7_1.tif'),
 PosixPath('../data/cellaltlas/images/19838_1252_F8_1.tif'),
 PosixPath('../data/cellaltlas/images/8346_22_C1_1.tif'),
 PosixPath('../data/cellaltlas/images/47032_977_G4_4.tif'),
 PosixPath('../data/cellaltlas/images/37367_517_E4_2.tif'),
 PosixPath('../data/cellaltlas/images/36268_407_B8_1.tif'),
 PosixPath('../data/cellaltlas/images/64554_1164_A6_2.tif'),
 PosixPath('../data/cellaltlas/images/60398_1596_E1_1.tif'),
 PosixPath('../data/cellaltlas/images/46658_784_B12_1.tif'),
 PosixPath('../data/cellaltlas/images/27897_273_C8_2.tif'),
 PosixPath('../data/cellaltlas/images/LICENSE.txt'),
 PosixPath('../data/cellaltlas/images/36268_404

In [29]:
export_folder = Path('../exports/')

for f in files:
    if f.suffix == '.tif':
        
        image = skimage.io.imread(f)
        m_in, m_out = nuclei_in_out(image)
        
        m_in_df = pd.DataFrame(m_in)
        m_out_df = pd.DataFrame(m_out)
        
        export_in = export_folder.joinpath(f.stem+'_in.csv')
        export_out = export_folder.joinpath(f.stem+'_out.csv')
        
        m_in_df.to_csv(export_in, index=False)
        m_out_df.to_csv(export_out, index=False)